In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

In [3]:
# Saving and Restoring Models
# Model progress can be saved during and after training.
# This allows a model to resume where it left off and to avoid
# unnecessarily long training times.
# Saving allows you to share your model and others can recreate 
# your work.
# Most ML practitioners share the following:
# 1. Code to create the model
# 2. Trained weights (parameters) for the model
# Be careful with untrusted code. TF models are code.

In [4]:
# Options:
# 1. tf.keras
# 2. TF Save and restore guide
# 3. TF Saving in eager execution mode

In [5]:
# Install and import TF and dependencies.
!pip install h5py pyyaml

You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
# Get an example dataset. We'll use the first 1000 examples 
# of the MNIST dataset.
from __future__ import absolute_import, division,\
    print_function, unicode_literals

In [7]:
import os
import tensorflow as tf
from tensorflow import keras

In [8]:
(train_images, train_labels), (test_images, test_labels) = \
    tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 5s 0us/step


In [9]:
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

In [10]:
train_images = train_images[:1000].reshape(-1, 28*28) / 255.0
test_images = test_images[:1000].reshape(-1, 28*28) / 255.0

In [12]:
# Define a short sequential model, just to demonstrate saving 
# and loading weights.
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, \
            activation=tf.keras.activations.relu,\
            input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, \
            activation=tf.keras.activations.softmax)
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(),\
        loss=tf.keras.losses.sparse_categorical_crossentropy,\
        metrics=['accuracy'])
    
    return model

In [13]:
model = create_model()
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Save checkpoints during training
# Train the model and pass it the ModelCheckpoint callback.
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [15]:
# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(\
                checkpoint_path, save_weights_only=True,\
                verbose=1)

In [16]:
# Create the model.
model = create_model()

In [17]:
# Train the model, passing the callback to training.
model.fit(train_images, train_labels, epochs=10, \
          validation_data=(test_images, test_labels),\
          callbacks=[cp_callback])

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 992/1000 [============================>.] - ETA: 0s - loss: 1.1333 - acc: 0.6562
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 4s 4ms/sample - loss: 1.1306 - acc: 0.6570 - val_loss: 0.7078 - val_acc: 0.7850
Epoch 2/10
 960/1000 [===========================>..] - ETA: 0s - loss: 0.4145 - acc: 0.8833
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.4147 - acc: 0.8840 - val_loss: 0.5269 - val_acc: 0.8370
Epoch 3/10
 960/1000 [===========================>..] - ETA: 0s - loss: 0.2845 - acc: 0.9250
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 2s 2ms/sample - loss: 0.2814 - acc: 0.9250 - val_loss: 0.4615 - val_acc: 0.8550
Epoch 4/10
 992/1000 [============================>.] - ETA: 0s - loss: 0.2178 - acc: 0.9476
Epoch 00004: saving model to training_1/cp.ckpt
1000/1000 [==

In [18]:
!ls {checkpoint_dir}

checkpoint		     cp.ckpt.data-00001-of-00002
cp.ckpt.data-00000-of-00002  cp.ckpt.index


In [19]:
# Create a new untrained model. When restoring a model from only
# weights, you must have a model with the same architecture as
# the original model. Since it has the same architecture, we can
# share weights, despite the fact that it's a different instance
# of the model.

In [20]:
model = create_model()

In [21]:
# Evaluate it on the test set.
loss, acc = model.evaluate(test_images, test_labels, verbose=2)

1000/1000 - 1s - loss: 2.2401 - acc: 0.1980


In [22]:
print('Untrained model, accuracy: {:5.2f}%'.format(100*acc))

Untrained model, accuracy: 19.80%


In [23]:
# Load the weights from the checkpoint and re-evaluate.
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4098 - acc: 0.8720
Restored model, accuracy: 87.20%


In [24]:
# Checkpoint callback provides several options to give the
# resulting checkpoints unique names and adjust the checkpointing
# frequency.

In [25]:
# Train a new model, and save uniquely named checkpoints once
# every 5 epochs.
# Include the epoch in the file name. (Uses 'str.format')
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [26]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(\
                    checkpoint_path, verbose=1, \
                    save_weights_only=True, \
                    # Save wts every 5 epochs\
                    period=5)

In [27]:
model = create_model()

In [28]:
model.save_weights(checkpoint_path.format(epoch=0))

In [29]:
model.fit(train_images, train_labels, epochs=50, \
          callbacks=[cp_callback],\
          validation_data=(test_images,test_labels),\
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [30]:
# Look at the resulting checkpoints and choose the latest one.
! ls {checkpoint_dir}

checkpoint			  cp-0025.ckpt.data-00001-of-00002
cp-0000.ckpt.data-00000-of-00002  cp-0025.ckpt.index
cp-0000.ckpt.data-00001-of-00002  cp-0030.ckpt.data-00000-of-00002
cp-0000.ckpt.index		  cp-0030.ckpt.data-00001-of-00002
cp-0005.ckpt.data-00000-of-00002  cp-0030.ckpt.index
cp-0005.ckpt.data-00001-of-00002  cp-0035.ckpt.data-00000-of-00002
cp-0005.ckpt.index		  cp-0035.ckpt.data-00001-of-00002
cp-0010.ckpt.data-00000-of-00002  cp-0035.ckpt.index
cp-0010.ckpt.data-00001-of-00002  cp-0040.ckpt.data-00000-of-00002
cp-0010.ckpt.index		  cp-0040.ckpt.data-00001-of-00002
cp-0015.ckpt.data-00000-of-00002  cp-0040.ckpt.index
cp-0015.ckpt.data-00001-of-00002  cp-0045.ckpt.data-00000-of-00002
cp-0015.ckpt.index		  cp-0045.ckpt.data-00001-of-00002
cp-0020.ckpt.data-00000-of-00002  cp-0045.ckpt.index
cp-0020.ckpt.data-00001-of-00002  cp-0050.ckpt.data-00000-of-00002
cp-0020.ckpt.index		  cp-0050.ckpt.data-00001-of-00002
cp-0025.ckpt.data-00000-of-00002  cp-0050.ckpt.index


In [31]:
latest = tf.train.latest_checkpoint(checkpoint_dir)

In [32]:
latest

'training_2/cp-0050.ckpt'

In [33]:
model = create_model()

In [34]:
model.load_weights(latest)

In [35]:
loss, acc = model.evaluate(test_images, test_labels, verbose=2)

1000/1000 - 1s - loss: 0.4841 - acc: 0.8790


In [36]:
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

Restored model, accuracy: 87.90%


In [37]:
# What are these files? The above code stores the weights to a 
# collection of checkpoint-formatted files containing only the
# trained weights in a binary format. Checkpoints contain:
# - One or more shards that contain your model's weights.
# - An index file that indicates which weights are stored in 
# which shard.
# If you are only training a model on a single machine, you'll
# have one shard with the suffix: .data-00000-of-00001

In [38]:
# Manually save weights.
model.save_weights('./checkpoints/my_checkpoint')

In [39]:
# Restore the weights
model = create_model()

In [40]:
model.load_weights('./checkpoints/my_checkpoint')

In [41]:
loss,acc = model.evaluate(test_images, test_labels, verbose=2)

1000/1000 - 1s - loss: 0.4841 - acc: 0.8790


In [42]:
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

Restored model, accuracy: 87.90%


In [43]:
# Save the entire model.
# The entire model can be saved to a file that contains:
# - the weight values
# - the model's configuration
# - the optimizer's configuration (depending on the setup)
# This allows you to checkpoint a model and resume training later,
# from the exact same state, without access to the original code.
# Fully-functional models can be loaded in TensorFlow.js and then
# trained and run in web browsers. They can also be converted to
# run on mobile devices using TFLite.

In [44]:
# Keras provides a basic save format using the HDF5 standard.
# For our purposes, the saved model can be treated as a single
# binary blob.
model = create_model()

In [45]:
model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 2s 2ms/sample - loss: 1.1559 - acc: 0.6740
Epoch 2/5
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.4431 - acc: 0.8720
Epoch 3/5
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.2883 - acc: 0.9240
Epoch 4/5
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.2126 - acc: 0.9400
Epoch 5/5
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.1596 - acc: 0.9620


In [46]:
# Save entire model to a HDF5 file.
model.save('my_model.h5')

In [47]:
# Recreate the exact same model, including weights and optimizer.
new_model = keras.models.load_model('my_model.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [48]:
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [49]:
# Check its accuracy.
loss,acc = new_model.evaluate(test_images, test_labels, verbose=2)

1000/1000 - 1s - loss: 0.4417 - acc: 0.8530


In [50]:
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

Restored model, accuracy: 85.30%


In [51]:
# Note: Keras saves models by inspecting the architecture.
# Currently, it is not able to save TF optimizers (from tf.train).
# When using those, you will need to re-compile the model after
# loading, and you will lose the state of the optimizer.

In [52]:
# This method of saving a tf.keras model is experimental and
# may change in future versions.
# Build a fresh model.
model = create_model()

In [53]:
# Train the model.
model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 2s 2ms/sample - loss: 1.1372 - acc: 0.6860
Epoch 2/5
1000/1000 [==============================] - 1s 951us/sample - loss: 0.4192 - acc: 0.8780
Epoch 3/5
1000/1000 [==============================] - 1s 974us/sample - loss: 0.2785 - acc: 0.9190
Epoch 4/5
1000/1000 [==============================] - 1s 963us/sample - loss: 0.2159 - acc: 0.9480
Epoch 5/5
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.1452 - acc: 0.9760


In [54]:
# Create a saved_model
import time
saved_model_path = "./saved_models/"+str(int(time.time()))

In [55]:
tf.contrib.saved_model.save_keras_model(model, saved_model_path)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: ['train']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in expor

In [56]:
# Look in the directory.
!ls {saved_model_path}

assets	saved_model.pb	variables


In [57]:
# Reload a fresh Keras model from the saved model.
new_model = tf.contrib.saved_model.load_keras_model(\
                    saved_model_path)

In [58]:
new_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [59]:
# Run the restored model.
# This model has to be compiled before evaluating.
# This step is not required if the saved model is only being
# deployed.
new_model.compile(optimizer=tf.keras.optimizers.Adam(),\
        loss=tf.keras.losses.sparse_categorical_crossentropy,\
        metrics=['accuracy'])

In [60]:
# Evaluate the restored model.
loss,acc = new_model.evaluate(test_images,test_labels,verbose=2)

1000/1000 - 1s - loss: 0.4293 - acc: 0.8640


In [61]:
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

Restored model, accuracy: 86.40%
